# Data Preparation, Pipelines & Model 

In [ ]:
# Modules importeren
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler, OneHotEncoder, FunctionTransformer, PowerTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import r2_score, mean_squared_error, root_mean_squared_error
from sklearn.compose import ColumnTransformer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.ensemble import RandomForestRegressor
from scipy.stats import randint
import joblib

In [ ]:
# Dataset importeren 
df = pd.read_csv("/Users/odessa/Desktop/Applied Data Science & AI/Data Science/Code Inleiding data science/song_data.csv")

In [ ]:
print(df['time_signature'].value_counts(dropna=False))

In [ ]:
df = df[df['time_signature'] > 1]

In [ ]:
# Target variabele maken 
target = 'song_popularity'

### Phase 3: Data Preparation

In [ ]:
# 2 nummers droppen
#df = df.drop([7119, 11171]).reset_index(drop=True)

In [ ]:
print(f"Totaal aantal waardes in de dataframe vóór het verwijderen van dubbele waardes uit song_name en song_duration_ms: {len(df)}")

# Dubbele waardes droppen van song_name en song_duration 
# Als ik alleen song_name duplicates zou verwijderen, zou ik misschien covers van nummers verwijderen, dus daarom check ik ook de song_duration 
df.drop_duplicates(subset=['song_name', 'song_duration_ms'], inplace = True)
print(f"Totaal aantal waardes in de dataframe na verwijderen van dubbele waardes uit song_name en song_duration_ms: {len(df)}")

In [ ]:
df.drop(columns=["song_name"], inplace=True) # inplace=True veranderd de originele dataframe zonder nieuwe dataframe te maken 

In [ ]:
X = df.drop(columns=[target], axis=1)
y = df[target]

In [ ]:
print(X['key'].unique())

In [ ]:
# Zorg dat 'key' en 'time_signature' als categorisch gezien worden
X['time_signature'] = X['time_signature'].astype(str)
X['key'] = X['key'].astype(int)  # blijft numeriek voor de cyclische encoder

## Winsorizer Class

In [ ]:
# BaseEstimator zorgt dat sklearn mijn class kan herkennen als model/stap in pipeline.
# TransformerMixin geeft .fit_transform().
class Winsorizer(BaseEstimator, TransformerMixin):
    def __init__(self, kolommen): 
        self.kolommen = kolommen 
        self.grenzen_ = None # '_' betekent dat het attribuut pas beschikbaar wordt, nadat fit() is uitgevoerd. 
                             # None, omdat de grenzen nog niet bestaan -- worden berekend bij fit().

    def fit(self, X, y=None):
        """Bereken de onder- en bovengrenzen per kolom met interkwartielafstand-regel."""
        self.grenzen_ = {}
        for kolom in self.kolommen:
            Q1 = X[kolom].quantile(0.25)
            Q3 = X[kolom].quantile(0.75)
            IKR = Q3 - Q1 
            ondergrens = Q1 - 1.5 * IKR
            bovengrens = Q3 + 1.5 * IKR
            self.grenzen_[kolom] = (ondergrens, bovengrens)
        return self 
    
    def transform(self, X):
        """Winsoriseer uitschieters: vervang alle waardes buiten de grenzen met de dichtstbijzijnde grenswaarde."""
        X = X.copy() # kopie maken van data
        for kolom, (ondergrens, bovengrens) in self.grenzen_.items():
            dtype = X[kolom].dtype

            # If the column is integer, cast the thresholds first
            if np.issubdtype(dtype, np.integer):
                ondergrens = int(round(ondergrens))
                bovengrens = int(round(bovengrens))

            X.loc[X[kolom] < ondergrens, kolom] = ondergrens
            X.loc[X[kolom] > bovengrens, kolom] = bovengrens

            # Just to be sure, cast back to the original dtype
            X[kolom] = X[kolom].astype(dtype)
        return X
    
    def get_feature_names_out(self, input_features=None):
        """Zorg dat de kolomnamen behouden blijven in de pipeline."""
        return np.array(self.kolommen)

## Key Cyclic Encoder class 

In [ ]:
class ToonsoortCyclischeEncoder(BaseEstimator, TransformerMixin):
    """
    Cyclisch encoden van de toonsoort/key (0-11) met sinus en cosinus. 
    """

    def __init__(self, kolom='key', max_waarde=12):
        self.kolom = kolom
        self.max_waarde = max_waarde
    
    def fit(self, X, y=None):
        return self 
    
    def transform(self, X):
        X = X.copy()
        k = X[self.kolom]
        X[f'{self.kolom}_sin'] = np.sin(2 * np.pi * k/self.max_waarde)
        X[f'{self.kolom}_cos'] = np.cos(2 * np.pi * k/self.max_waarde)
        return X.drop(columns=[self.kolom])
    
    def get_feature_names_out(self, input_features=None):
        return np.array([f'{self.kolom}_sin', f'{self.kolom}_cos'])

In [ ]:
def get_feature_names_from_column_transformer(ct):
    """Haalt feature names uit een ColumnTransformer, ook als nested pipelines aanwezig zijn."""
    output_features = []

    for name, transformer, cols in ct.transformers_:
        if transformer == 'drop':
            continue
        elif transformer == 'passthrough':
            output_features.extend(cols)
        elif hasattr(transformer, 'get_feature_names_out'):
            try:
                names = transformer.get_feature_names_out(cols)
                output_features.extend(names)
            except:
                output_features.extend(cols)
        elif hasattr(transformer, 'transformers_'):  # nested ColumnTransformer
            output_features.extend(get_feature_names_from_column_transformer(transformer))
        elif isinstance(transformer, Pipeline):
            last_step = transformer.steps[-1][1]
            if hasattr(last_step, 'get_feature_names_out'):
                try:
                    names = last_step.get_feature_names_out(cols)
                    output_features.extend(names)
                except:
                    output_features.extend(cols)
            else:
                output_features.extend(cols)
        else:
            output_features.extend(cols)
    return output_features

### Phase 4: Modeling 

Supervised learning, omdat je de uitkomst al hebt 
<br>
Supervised learning heeft 2 hoofdtakken: regressie en classificatie 
<br>
RMSE 
<br>
Meervoudige lineare regressie 
<br>
Logistieke lineare regressie is classification 
<br>
Random forests is het begin van dat machine learning slim werd 

In [ ]:
def nieuwe_features(df):
    df = df.copy()


    df['energy_dance'] = df['energy'] * df['danceability']
    df['valence_dance'] = df['audio_valence'] * df['danceability']

    #Ratio featurres
    df['acoustic_dance_ratio'] = df['acousticness'] / (df['danceability'] + 0.001)
    df['duration_energy_ratio'] = df['song_duration_ms'] / (df['energy'] + 0.001)

    # Replace any infinities that could appear from division
    df.replace([np.inf, -np.inf], np.nan, inplace=True)
    df.fillna(0, inplace=True)
    return df
feature_engineering = FunctionTransformer(nieuwe_features, validate=False)

In [ ]:
# Kolommen indelen
kolommen_winsoriseren = ['song_duration_ms', 'tempo']
categorische_kolommen = ['time_signature']
cyclische_kolommen = ['key']
engineered_features = [
    'energy_dance', 'valence_dance',
    'acoustic_dance_ratio', 'duration_energy_ratio'
]

# Alle numerieke kolommen behalve target
numerieke_kolommen = X.select_dtypes(include=['int64', 'float64']).columns.tolist()

# Alle numerieke kolommen behalve target
overige_kolommen = [
    c for c in numerieke_kolommen
    if c not in kolommen_winsoriseren + cyclische_kolommen + categorische_kolommen
]

In [ ]:
# Train en test set maken 
X_train, X_test, y_train, y_test = train_test_split(
   X, y, test_size=0.2, random_state = 42
)

In [ ]:
print("Numeriek:", numerieke_kolommen)
print("Overige:", overige_kolommen)

In [ ]:
skewed_cols = ['acousticness', 'instrumentalness', 'liveness', 'loudness', 'speechiness']

## Preprocessors

In [ ]:
# Preprocessor voor Lineaire Regressie
preprocessor = ColumnTransformer([
    # 1. outliers winsoriseren 
    ('winsor_scale', Pipeline([
        ('winsor', Winsorizer(kolommen=kolommen_winsoriseren)),
        ('scaler', StandardScaler())
    ]), kolommen_winsoriseren),

    # 2. yeo-johnson transform
    ('transform_scale', Pipeline([
        ('power', PowerTransformer(method='yeo-johnson')),
        ('scaler', StandardScaler())
    ]), ['danceability', 'audio_valence', 'energy']),

    # 3. Log1p transform 
    ('log_scale', Pipeline([
        ('log', FunctionTransformer(np.log1p, validate=False)),
        ('scaler', StandardScaler())
    ]), ['acoustic_dance_ratio', 'duration_energy_ratio']),

    # 4. One-hot encoder
    ('onehot', OneHotEncoder(handle_unknown='ignore'), categorische_kolommen),

    # 5. Cyclisch encoden Key 
    ('key_cyclisch', Pipeline([
        ('encoder', ToonsoortCyclischeEncoder(kolom='key', max_waarde=12)),
        ('scaler', StandardScaler())
    ]), cyclische_kolommen),

    # 6. Overige numerieke waardes scalen 
    ('scale_rest', StandardScaler(), 
      [c for c in overige_kolommen 
        if c not in ['danceability', 'audio_valence', 'energy', 'loudness']] 
        + ['energy_dance', 'valence_dance'])
],
remainder='drop')

In [ ]:
# Preprocessor voor Random Forest
preprocessor_rf = ColumnTransformer([
    ('winsor', Winsorizer(kolommen=kolommen_winsoriseren), kolommen_winsoriseren),
    ('onehot', OneHotEncoder(handle_unknown='ignore'), categorische_kolommen),
    ('key_cyclisch', ToonsoortCyclischeEncoder(kolom='key', max_waarde=12), cyclische_kolommen)
], remainder='passthrough')

## Pipelines

In [ ]:
# Lineaire Regressie pipeline 
lineair_pipeline = Pipeline([
    ('feature_creation', feature_engineering),
    ('preprocess', preprocessor),
    ('model', LinearRegression())
])

In [ ]:
# Random forest pipeline
random_forest_pipeline = Pipeline([
    ('feature_creation', feature_engineering),
    ('preprocess', preprocessor_rf),
    ('model', RandomForestRegressor(random_state=42, n_jobs=-1))
])

In [ ]:
# Trainen en evalueren 
lineair_pipeline.fit(X_train, y_train)
y_pred = lineair_pipeline.predict(X_test)

In [ ]:
# Apply only preprocessing (exclude model)
X_transformed = lineair_pipeline[:-1].fit_transform(X)

# Get all feature names robustly
feature_names = get_feature_names_from_column_transformer(
    lineair_pipeline.named_steps['preprocess']
)

# Make DataFrame
df_transformed = pd.DataFrame(X_transformed, columns=feature_names)

# Plot histograms
df_transformed.hist(figsize=(16, 12), bins=30)
plt.tight_layout()
plt.show()

In [ ]:
# Functie Skewness en kurtosis berekenen en highlighten van hoog scheef, matig scheef en redelijk symmetrisch 
def skewness_kurtosis(df):
    skewness = df.skew(numeric_only=True)
    kurtosis = df.kurt(numeric_only=True)

    stats_skew_kurt = pd.DataFrame({
        'skewness': skewness,
        'kurtosis': kurtosis
    })

    def highlighter(rij):
        skew = rij['skewness']
        color = ''
        if skew > 1 or skew < -1: # Hoge scheefheid (groter dan 1 of kleiner dan -1) kleur = rood 
            color = 'background-color: red; color: black;'
        elif 0.5 < abs(skew) <=1: # Matige scheefheid (scheefheid tussen -1 en -0.5 of tussen 0.5 en 1) kleur = blauw 
            color = 'background-color: #336df5; color: black;'
        elif abs(skew) <= 0.5: # Redelijk symmetrische distributie (scheefheid tussen -0.5 en 0.5) geen highlight
            color = 'color: white;'
        return ['' if c != 'skewness' else color for c in stats_skew_kurt.columns]
    
    styled = stats_skew_kurt.style.apply(highlighter, axis=1)
    display(styled)
    return stats_skew_kurt

In [ ]:
stats = skewness_kurtosis(X)
skewed_cols = stats.loc[stats['skewness'].abs() > 1].index.tolist()
print("Sterk scheve kolommen:", skewed_cols)

In [ ]:
stats = skewness_kurtosis(df_transformed)
skewed_cols = stats.loc[stats['skewness'].abs() > 1].index.tolist()
print("Sterk scheve kolommen:", skewed_cols)

## Fase 1: RandomizedSearchCV

In [ ]:
# Broad parameter ranges
param_dist = {
    'model__n_estimators': randint(100, 400),
    'model__max_depth': randint(4, 16),
    'model__min_samples_split': randint(2, 10),
    'model__min_samples_leaf': randint(1, 5)
}

random_search = RandomizedSearchCV(
    estimator=random_forest_pipeline,
    param_distributions=param_dist,
    n_iter=25,          # number of random combinations to test
    scoring='r2',
    cv=3,               # fewer folds = faster
    n_jobs=-1,
    random_state=42,
    verbose=2
)

random_search.fit(X_train, y_train)

print("Best parameters (random search):", random_search.best_params_)
print("Best CV R²:", random_search.best_score_)

# (Optional) Save for later
import joblib
joblib.dump(random_search, "random_search_rf.pkl")

# Store the best params for grid refinement
best = random_search.best_params_

Best parameters (random search): {'model__max_depth': 8, 'model__min_samples_leaf': 1, 'model__min_samples_split': 8, 'model__n_estimators': 373}
Best CV R²: 0.047162919920938094

## Fase 2: GridSearchCV 

In [ ]:
# Define a tighter grid around the best RandomizedSearchCV result
param_grid = {
    'model__n_estimators': [best['model__n_estimators'] - 50,
                            best['model__n_estimators'],
                            best['model__n_estimators'] + 50],
    'model__max_depth': [best['model__max_depth'] - 2,
                         best['model__max_depth'],
                         best['model__max_depth'] + 2],
    'model__min_samples_split': [best['model__min_samples_split'] - 1,
                                 best['model__min_samples_split'],
                                 best['model__min_samples_split'] + 1],
    'model__min_samples_leaf': [best['model__min_samples_leaf'] - 1,
                                best['model__min_samples_leaf'],
                                best['model__min_samples_leaf'] + 1]
}

# Remove invalid (≤ 0) values
for key, vals in param_grid.items():
    param_grid[key] = [v for v in vals if v > 0]

# Define grid search
grid_search = GridSearchCV(
    estimator=random_forest_pipeline,
    param_grid=param_grid,
    scoring='r2',
    cv=5,
    n_jobs=-1,
    verbose=2
)

grid_search.fit(X_train, y_train)

print("Best parameters (grid refinement):", grid_search.best_params_)
print("Cross-val R²:", grid_search.best_score_)

# Save the trained grid search for later use
import joblib
joblib.dump(grid_search, "grid_search_rf.pkl")

Best parameters (grid refinement): {'model__max_depth': 8, 'model__min_samples_leaf': 2, 'model__min_samples_split': 8, 'model__n_estimators': 373}
<br>
Cross-val R²: 0.05182079174157097

In [ ]:
grid_search = joblib.load("grid_search_rf.pkl")

In [ ]:
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import make_scorer, r2_score
import numpy as np

# Define KFold (5 splits, shuffled for randomness)
kfold = KFold(n_splits=5, shuffle=True, random_state=42)

# Evaluate Linear Regression
scores_lin = cross_val_score(lineair_pipeline, X, y, cv=kfold,
                             scoring=make_scorer(r2_score))

# Evaluate Random Forest
scores_rf = cross_val_score(random_forest_pipeline, X, y, cv=kfold,
                            scoring=make_scorer(r2_score))

print("Linear Regression R² per fold:", scores_lin)
print("Random Forest R² per fold:", scores_rf)
print("Mean ± SD (Linear):", np.mean(scores_lin), "±", np.std(scores_lin))
print("Mean ± SD (Random Forest):", np.mean(scores_rf), "±", np.std(scores_rf))


# Custom scorer for RMSE
rmse_scorer = make_scorer(lambda y_true, y_pred: 
                          root_mean_squared_error(y_true, y_pred),
                          greater_is_better=False)

# Add RMSE cross-val
scores_rf_rmse = cross_val_score(random_forest_pipeline, X, y, cv=kfold, scoring=rmse_scorer)
print("Random Forest RMSE per fold:", -scores_rf_rmse)


In [ ]:
from scipy.stats import ttest_rel

# Perform a paired t-test on the cross-validation R² scores
t_stat, p_val = ttest_rel(scores_rf, scores_lin)

print(f"T-statistic = {t_stat:.4f}, p-value = {p_val:.4f}")

# Interpretation
if p_val < 0.05:
    print("Significant difference: Random Forest and Linear Regression perform differently.")
else:
    print("No statistically significant difference detected.")

## Evaluatie test data

In [ ]:
beste_model = grid_search.best_estimator_
y_pred_rf = beste_model.predict(X_test)

r2_rf = r2_score(y_test, y_pred_rf)
rmse_rf = root_mean_squared_error(y_test, y_pred_rf)

print(f"Test R²: {r2_rf:.4f}")
print(f"Test RMSE: {rmse_rf:.4f}")

## Resultaten vergelijken

In [ ]:
y_pred_lin = lineair_pipeline.predict(X_test)
r2_lin = r2_score(y_test, y_pred_lin)
rmse_lin = root_mean_squared_error(y_test, y_pred_lin)

resultaten = pd.DataFrame({
    'Model': ['Lineaire regressie', 'Random Forest'],
    'R2': [r2_lin, r2_rf],
    'RMSE': [rmse_lin, rmse_rf]
}).round({'R2': 4, 'RMSE': 2})

display(resultaten)

### Observaties resultaten 

Random forest presteert beter dan lineare regressie: R² iets omhoog en RMSE iets omlaag. 
<br>
RMSE van 20 is logisch, omdat je populariteit niet alleen op basis van audio features kunt voorspellen. 

### Feature Importances

In [ ]:
model = grid_search.best_estimator_.named_steps['model']
importances = model.feature_importances_

# Use helper to get feature names
preprocessor = grid_search.best_estimator_.named_steps['preprocess']
feature_names = get_feature_names_from_column_transformer(preprocessor)

# Create series and sort 
feat_importance = pd.Series(importances, index=feature_names).sort_values(ascending=False)

#Plot top 15 belangrijke features

plt.figure(figsize=(10, 6))
feat_importance.head(15).plot(kind='bar', color='mediumseagreen')
plt.title("Belangrijkste Features (Random Forest)")
plt.ylabel("Feature Importance")
plt.xlabel("Feature")
plt.tight_layout()
plt.show()

In [ ]:
print(feat_importance.to_string())